In [70]:
# coding=utf-8
import sys
import numpy as np
import requests
import os
import pandas as pd
import csv
import sqlite3 as lite
from bs4 import BeautifulSoup
from selenium import webdriver
from shutil import copyfile
from ast import literal_eval

# Константы
all_words_links_path = 'all_words_links.npy'
vsekidki_fullwords_csv = 'vsekidki_fullwords.csv'
vsekidki_appwords_csv = "vsekidki_appwords.csv"
vsekidki_words_page = 'vsekidki_words_npy/vsekidki_words_page'
length = 45
global length

## Сделать all_words_links = [ссылки на все слова] (загрузить с диска)

In [26]:
if os.path.isfile(all_words_links_path):
    all_words_links = np.load(all_words_links_path).tolist()
else:
    vsekidki_url = "http://vsekidki.ru/word/page/"
    all_words_links = []
    for i in range(1, 45):
        r = requests.get(vsekidki_url + str(i) + '/')
        data = r.text
        soup = BeautifulSoup(data, "lxml")
        table_of_words = soup.find_all('a', class_='catalogue-heading-link')
        for link in table_of_words:
            all_words_links.append(link['href'])
    len(all_words_links)
    np.save(all_words_links_path, all_words_links)

## Основные функции

In [132]:
def Text_from_vsekidki_word(text):
    start = text.find("<")
    finish = text.find(">")
    text = text[:start] + text[finish + 1:]
    print("****11111****", text)
    check_for_tag = text.find('<', start)
    while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
    text = text[:int(len(text)/2)]
    return text

def print_word(word):
    printing = word['name'] + "\n    ЗНАЧЕНИЕ: " + word['definition'].strip() + "\n    ПРОИСХОЖДЕНИЕ: " + word['origin'].strip() + "\n\n" 
    print(printing)
    for i in range(len(word['fulltext'])):
        print(i, word['fulltext'][i])
        
def find_quote(soup):
    quote = ""
    try:
        quote = soup.find('div', class_='quote').text
        quote = quote.replace("\'\'", "\"").replace(",,", "\"")
    except Exception as msg:
        quote = ''
    return quote

def make_origin(quote):
    origin = ""
    try:
        origin = article_body.contents[4].text
    except:
        try:
            origin = article_body.contents[4]
        except:
            origin = ''
    return (origin + quote).encode('iso-8859-1').decode('utf-8').strip()

def get_fulltext_from_article_body(article_body):
    fulltext = []
    for item in article_body.contents[2:]:
        utf_str = ""
        try:
            utf_str = item.text.strip().encode('iso-8859-1').decode('utf-8')
        except:
            try:
                utf_str = item.strip().encode('iso-8859-1').decode('utf-8')
            except:
                try:
                    utf_str = item.strip()
                except:
                    utf_str = ""
        if utf_str != "":
            fulltext.append(utf_str)
    return fulltext

def find_name(soup):
    name = ""
    try:
        name = soup.find('h1', class_='new-title').text.encode('iso-8859-1').decode('utf-8').strip()
    except:
        name = soup.find('h1', class_='new-title').text
    return name

def make_appwords_file(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name','definition','type','group','examples','origin','hashtags','synonyms'))
        
nthng = "_"        
        
def add_appwords_to_file(dict, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dict:
            writer.writerow((word['name'],word['definition'],"","","",word['origin'],"",nthng))
  
def make_fullwords_file(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name','fulltext'))


def add_fullwords_to_file(dict, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dict:
            writer.writerow((word['name'], word['fulltext']))

## Распарсить pages from 1 to length, поместить их в vsekidki_words_page(i).npy

In [179]:
for i in range(1, length):
    # to rewrite all files delete "not"
    if not os.path.isfile(vsekidki_words_page + str(i) + '.npy'):
        words = []
        print(" PAGE ", i)
        for word_link in all_words_links[(i - 1)*30: i*30 + 1]:
            r = requests.get(word_link)
            soup = BeautifulSoup(r.text, "lxml")
            article_body = soup.find('div', itemprop='articleBody')
            name = find_name(soup)
            definition = article_body.contents[2].encode('iso-8859-1').decode('utf-8').strip()
            quote = find_quote(soup)
            origin = make_origin(quote=quote)
            fulltext = get_fulltext_from_article_body(article_body)
            word = {"name": name,
                    "origin": origin,
                    "definition": definition,
                    "fulltext": fulltext}
            words.append(word)
        dict_path = vsekidki_words_page + str(i) + '.npy'
        np.save(dict_path, words)
print("*********\n ГОТОВО\n*********")

for i in range(0, length):
    # to rewrite all files delete "not"
    if not os.path.isfile(vsekidki_words_page + str(i) + '.npy'):
        words = []
        print(" PAGE ", i)
        for j in range(i*30, (i + 1)*30):
            word = {"name": all_words[j]['name'],
                    "origin": all_words[j]['origin'],
                    "definition": all_words[j]['definition'],
                    "fulltext": all_words[j]['fulltext']}
            words.append(word)
        dict_path = vsekidki_words_page + str(i) + '.npy'
        np.save(dict_path, words)
print("*********\n ГОТОВО\n*********")


*********
 ГОТОВО
*********
 PAGE  1
 PAGE  2
 PAGE  3
 PAGE  4
 PAGE  5
 PAGE  6
 PAGE  7
 PAGE  8
 PAGE  9
 PAGE  10
 PAGE  11
 PAGE  12
 PAGE  13
 PAGE  14
 PAGE  15
 PAGE  16
 PAGE  17
 PAGE  18
 PAGE  19
 PAGE  20
 PAGE  21
 PAGE  22
 PAGE  23
 PAGE  24
 PAGE  25
 PAGE  26
 PAGE  27
 PAGE  28
 PAGE  29
 PAGE  30
 PAGE  31
 PAGE  32
 PAGE  33
 PAGE  34
 PAGE  35
 PAGE  36
 PAGE  37
 PAGE  38
 PAGE  39
 PAGE  40
 PAGE  41
 PAGE  42
 PAGE  43
 PAGE  44
*********
 ГОТОВО
*********


In [180]:
for i in range(len(all_words1)):
    print(i, all_words1[i]['name'])

0 Факбой Fuccboi
1 Факбой Fuccboi
2 Вофлот
3 Я шарю
4 Ауф
5 Вписка
6 Шкура
7 Тамблер герл
8 Стафф
9 Марамойка
10 ОТП
11 69
12 Амахасла
13 Люмпен (Lumpen)
14 Мудак
15 Вайпер
16 Жиза
17 Трип, триповать
18 Довен
19 Крипово, Крипота
20 Баребух
21 ОРУ
22 Ц
23 Цп в ЛС - Цопе, ЦП, Цопэ
24 Шипперить
25 Тян
26 Шарит
27 Лакшери
28 Гомункул
29 Синий кит / Задание синего кита
30 Сасный
31 Калич
32 Калич
33 Лабутены
34 Сэдбой
35 ПТН ПНХ
36 Лампово
37 Харвест
38 Репит
39 Сканка с зипом
40 4:20
41 Чоп-чоп
42 Свитшот
43 ЧСВ
44 ТП
45 Эдлиб
46 Хизи
47 Вейпшоп
48 Челлендж
49 Кпуала
50 Солевая тян
51 Охаё, охайо
52 Бадман, бэдман
53 Шер, шерить
54 ЛП
55 Зольд (зольды, зольдберг)
56 Палпинки
57 Оксимирон
58 Личинус
59 Кантик
60 Уцы, уцышка
61 Бэкграунд
62 По фишке (армейское выражение)
63 По фишке (армейское выражение)
64 322
65 Потрачено
66 Кринж
67 Кк
68 Хуй жопа
69 Мазафака, мазафакер
70 Рин рин пуси
71 Азаза
72 Шавуха
73 Шпак, шпануть
74 Бэнч
75 Пипидастры
76 Челендж
77 Мажор
78 Лолировать
79 Сарт
80 С

In [181]:
for i in range(len(all_words)):
    print(i, all_words[i]['name'])

0 АУЕ
1 Патимейкер
2 Аксемора
3 Голландский штурвал
4 Трэшер (Thrasher)
5 2к17, 2к16, 2к15
6 Дратуте
7 Зашквар
8 Флекс, Флексить, низкий флекс
9 Рофл, Рофлить
10 Стоник
11 Вейпер
12 Хайповый, Хайп, Хайпим
13 Эба
14 Сап двач
15 Оптимус ганг
16 Изи
17 Пососи писос
18 Краш
19 Социоблядь, социблядь
20 Эээ блэт туох дьиигин дээ
21 Куни
22 По фану
23 Шерь
24 Кек
25 15 год (пятнадцатый год)
26 Скр, Skrrt, скёр, скер
27 Пинрол
28 Байтить, Байт
29 Чепушило
30 Факбой Fuccboi
31 Факбой Fuccboi
32 Вофлот
33 Я шарю
34 Ауф
35 Вписка
36 Шкура
37 Тамблер герл
38 Стафф
39 Марамойка
40 ОТП
41 69
42 Амахасла
43 Люмпен (Lumpen)
44 Мудак
45 Вайпер
46 Жиза
47 Трип, триповать
48 Довен
49 Крипово, Крипота
50 Баребух
51 ОРУ
52 Ц
53 Цп в ЛС - Цопе, ЦП, Цопэ
54 Шипперить
55 Тян
56 Шарит
57 Лакшери
58 Гомункул
59 Синий кит / Задание синего кита
60 Сасный
61 Калич
62 Калич
63 Лабутены
64 Сэдбой
65 ПТН ПНХ
66 Лампово
67 Харвест
68 Репит
69 Сканка с зипом
70 4:20
71 Чоп-чоп
72 Свитшот
73 ЧСВ
74 ТП
75 Эдлиб
76 Хизи
7

## сделать all_words - массив всех слов

In [178]:
all_words1 = []
for i in range(1, length):
    read_dictionary = np.load(vsekidki_words_page + str(i) + '.npy').tolist()
    all_words1 += read_dictionary   
print("Слов:", len(all_words1))

Слов: 1320


In [176]:
print("Слов:", len(all_words))

Слов: 1350


In [168]:
all_words1[-100:]

[{'definition': '',
  'fulltext': ['Внимание!',
   'Слово',
   'Блатата',
   'находится в категории "Объясните людям?" и требуется его значение. Если Вы знаете, откуда пошло "Блатата" или что это значит, пожалуйста напишите:',
   '+ Написать значение слова',
   "function openbox(id){\n    display = document.getElementById(id).style.display;\n\n    if(display=='none'){\n       document.getElementById(id).style.display='block';\n    }else{\n       document.getElementById(id).style.display='none';\n    }\n}"],
  'name': 'Блатата (Требуется значение слова)',
  'origin': 'Слово'},
 {'definition': 'Апа- тетя по-татарски. Апашка- творческий подход.',
  'fulltext': ['Апа- тетя по-татарски. Апашка- творческий подход.'],
  'name': 'Апашка',
  'origin': ''},
 {'definition': 'Сник пик это нарезка нарезка самых крутых и захватывающих кадров из фильма для привлечения внимания к фильму. Обычно такая нарезка присутствует в трейлерах и бывает дополняется моментами, которых и не будет в фильме.',
  'ful

In [167]:
all_words[-100:]

[{'definition': 'Ясн это недописанное слово "Ясно", означает соответственно то, что человеку все понятно.',
  'fulltext': ['Ясн это недописанное слово "Ясно", означает соответственно то, что человеку все понятно.'],
  'name': 'Ясн',
  'origin': ''},
 {'definition': 'ССЖВ - смерть сукам жизнь (волю) ворам. Это сокращение.',
  'fulltext': ['ССЖВ - смерть сукам жизнь (волю) ворам. Это сокращение.'],
  'name': 'ССЖВ',
  'origin': ''},
 {'definition': 'С.С.Ж.В. Смерть Сукам Жизнь Ворам. Пошло от фразы АУЕ и говорят обычно так школьники или же совсем не серьезные люди.',
  'fulltext': ['С.С.Ж.В. Смерть Сукам Жизнь Ворам. Пошло от фразы АУЕ и говорят обычно так школьники или же совсем не серьезные люди.',
   'Источник:',
   'http://vsekidki.ru/217-aue.html'],
  'name': 'Ж.В.С.С.',
  'origin': 'Источник:'},
 {'definition': 'Кеблы - каблы. Переводятся как каблуки.',
  'fulltext': ['Кеблы - каблы. Переводятся как каблуки.',
   'Пример текста:',
   'купила сегодня новые кеблы.',
   'dle_flash_beg

## Сохранить all_words в vsekidki_dict.csv

In [133]:
make_fullwords_file(vsekidki_fullwords_csv)
add_fullwords_to_file(all_words, vsekidki_fullwords_csv)

make_appwords_file(vsekidki_appwords_csv)
add_appwords_to_file(all_words, vsekidki_appwords_csv)

fullwords_df = pd.read_csv(vsekidki_fullwords_csv)
appwords_df = pd.read_csv(vsekidki_appwords_csv)

In [134]:
appwords_df.head(3)

,name,definition,type,group,examples,origin,hashtags,synonyms
0,АУЕ,А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. ...,NaN,NaN,NaN,"Воряги так говорят, переводится как АРЕСТАНСКИ...",NaN,_
1,Патимейкер,Новая песня с незамысловатым текстом. Автор - ...,NaN,NaN,NaN,"Патимейкеры это те, кто организовывают тусу (д...",NaN,_
2,Аксемора,"Аксиомора ошибочное произнесение слова ""оксюмо...",NaN,NaN,NaN,"Компания ""Мегафон"" переозвучила свою рекламу с...",NaN,_


In [135]:
fullwords_df.head(3)

,name,fulltext
0,АУЕ,['А.У.Е расшифровка: Жизнь Ворам Сметь мусорам...
1,Патимейкер,['Новая песня с незамысловатым текстом. Автор ...
2,Аксемора,"['Аксиомора ошибочное произнесение слова ""оксю..."


In [136]:
fullwords_df['fulltext'][12]

"['Хайповый', 'означает модный, в теме, шарит что происходит в мире моды.', 'Хайп', 'это то, что сейчас модно.', 'Хайпим', 'означает тусим, развлекаемся, зажигаем.', 'Например:', 'Гоша Рубчинский хайп! (означает, что Гоша Рубчинский сейчас в моде)']"

In [60]:
pkl_file = "pkl_file.pkl"
fullwords_df.to_pickle(path=pkl_file)

In [61]:
fullwords_pkl_df = pd.read_pickle(path=pkl_file)

In [62]:
fullwords_pkl_df.head(13)

,name,fulltext
0,АУЕ,['А.У.Е расшифровка: Жизнь Ворам Сметь мусорам...
1,Патимейкер,['Новая песня с незамысловатым текстом. Автор ...
2,Аксемора,"['Аксиомора ошибочное произнесение слова ""оксю..."
3,Голландский штурвал,['Один берет руку другого и начинает управлять...
4,Трэшер (Thrasher),"['Thrasher — трэшер это скейтерский бренд, кот..."
5,"2к17, 2к16, 2к15","['2к16 это 2016, то есть год. Букву', 'к', 'пи..."
6,Дратуте,"['Дратути - это приветствие (здравствуйте, доб..."
7,Зашквар,['Самое класическое и актуальное и по сей день...
8,"Флекс, Флексить, низкий флекс","['Танцевать под качающий бит, музыку. Обычно н..."
9,"Рофл, Рофлить",['Рофл - Очень громко смеяться или хохотать. Р...


In [92]:
all_words_df = pd.DataFrame(all_words)
all_words_df.head(3)

,definition,fulltext,name,origin
0,А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. ...,"[, Не пропускайте новые модные слова и мемы: В...",АУЕ,"Воряги так говорят, переводится как АРЕСТАНСКИ..."
1,Новая песня с незамысловатым текстом. Автор - ...,"[, Не пропускайте новые модные слова и мемы: В...",Патимейкер,"Патимейкеры это те, кто организовывают тусу (д..."
2,"Аксиомора ошибочное произнесение слова ""оксюмо...","[, Не пропускайте новые модные слова и мемы: В...",Аксемора,"Компания ""Мегафон"" переозвучила свою рекламу с..."


In [97]:
all_words_df['fulltext'] = all_words_df['fulltext'].apply(remake_fulltext_arrays)

In [122]:
print(type(all_words[0]['fulltext']))

<class 'list'>


In [123]:
for item in all_words:
    item['fulltext'] = remake_fulltext_arrays(item['fulltext'])

In [124]:
print(all_words[0]['fulltext'])

['А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. Крик, одобрение, кричалка. Что то типо асаб, но новый уровень!', 'Воряги так говорят, переводится как АРЕСТАНСКИЙ УКЛАД ЕДИН ИЛИ АРЕСТАНСКОЕ УРКАГАНСКОЕ ЕДИНСТВО. Одним словом, АУЕ это воровское сообщество, которое против законов и ментов.', 'Значение 2:', 'А так же английское AUE означает приветсвие. АУЕ на латинице переводится как "Здравствуй!".', 'Уточнение:', 'АУЕ - используется у гопников как один из видов приветствия или одобрения .', 'Также используется у студентов как выражение бурной эмоции. (Oh yeahhhh)']


In [93]:
all_words_df['fulltext'][0]

['', 'Не пропускайте новые модные слова и мемы: Вступайте в группу Вконтакте: https://vk.com/slang_me.', 'А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. Крик, одобрение, кричалка. Что то типо асаб, но новый уровень!', '', 'Воряги так говорят, переводится как АРЕСТАНСКИЙ УКЛАД ЕДИН ИЛИ АРЕСТАНСКОЕ УРКАГАНСКОЕ ЕДИНСТВО. Одним словом, АУЕ это воровское сообщество, которое против законов и ментов.', '', '', 'Значение 2:', 'А так же английское AUE означает приветсвие. АУЕ на латинице переводится как "Здравствуй!".', '', '', 'Уточнение:', 'АУЕ - используется у гопников как один из видов приветствия или одобрения .', '', 'Также используется у студентов как выражение бурной эмоции. (Oh yeahhhh)']


In [111]:
all_words[0]

{'definition': 'А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. Крик, одобрение, кричалка. Что то типо асаб, но новый уровень!',
 'fulltext': ['',
  'Не пропускайте новые модные слова и мемы: Вступайте в группу Вконтакте: https://vk.com/slang_me.',
  'А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. Крик, одобрение, кричалка. Что то типо асаб, но новый уровень!',
  '',
  'Воряги так говорят, переводится как АРЕСТАНСКИЙ УКЛАД ЕДИН ИЛИ АРЕСТАНСКОЕ УРКАГАНСКОЕ ЕДИНСТВО. Одним словом, АУЕ это воровское сообщество, которое против законов и ментов.',
  '',
  '',
  'Значение 2:',
  'А так же английское AUE означает приветсвие. АУЕ на латинице переводится как "Здравствуй!".',
  '',
  '',
  'Уточнение:',
  'АУЕ - используется у гопников как один из видов приветствия или одобрения .',
  '',
  'Также используется у студентов как выражение бурной эмоции. (Oh yeahhhh)'],
 'name': 'АУЕ',
 'origin': 'Воряги так говорят, переводится как АРЕСТАНСКИЙ УКЛАД ЕДИН ИЛИ АРЕСТАНСКОЕ УРКАГАНСКОЕ ЕДИНСТВО. Одним сло

In [108]:
type(all_words)

list

In [109]:
type(all_words[0])

dict

In [77]:
fullwords_df['fulltext'][5]

"['2к16 это 2016, то есть год. Букву', 'к', 'пишут вместо нуля.', '', '', 'Пример выражения:', '', '- найк аир макс зашквар в 2к16!', '', '- NBA 2К16 (игра нба 2016)', '', '- это уже не актуально в 2к16', '', '', 'dle_media_begin:https://www.youtube.com/watch?v=AfN8a8wX7YM', '', 'dle_media_end', '', '', 'Можно так же изменять цифру и писать разные года.', '', '', 'dle_media_begin:https://www.youtube.com/watch?v=5PrUUV9la8o', '', 'dle_media_end', '', '', 'Уточнение: Буква k в сочетании с числом обозначает кратную приставку кило-, т.е. множитель 1000. Например: warhammer 40000 иногда записывается, как warhammer 40k, или w40k. (Игорь,Витебск)', '', '', 'Уточнение: Это когда человек шарит Уточнение: Это когда ты решил стать слишком модным даже для цифр.', '', '', 'Уточнение: к расшифровывается как косарь, тысяча. две тысячи (два косаря) шестнадцатый. (Вася ,спб).']"

In [79]:
old_str = "['2к16 это 2016, то есть год. Букву', 'к', 'пишут вместо нуля.', '', '', 'Пример выражения:', '', '- найк аир макс зашквар в 2к16!', '', '- NBA 2К16 (игра нба 2016)', '', '- это уже не актуально в 2к16', '', '', 'dle_media_begin:https://www.youtube.com/watch?v=AfN8a8wX7YM', '', 'dle_media_end', '', '', 'Можно так же изменять цифру и писать разные года.', '', '', 'dle_media_begin:https://www.youtube.com/watch?v=5PrUUV9la8o', '', 'dle_media_end', '', '', 'Уточнение: Буква k в сочетании с числом обозначает кратную приставку кило-, т.е. множитель 1000. Например: warhammer 40000 иногда записывается, как warhammer 40k, или w40k. (Игорь,Витебск)', '', '', 'Уточнение: Это когда человек шарит Уточнение: Это когда ты решил стать слишком модным даже для цифр.', '', '', 'Уточнение: к расшифровывается как косарь, тысяча. две тысячи (два косаря) шестнадцатый. (Вася ,спб).']"
new_str = literal_eval(old_str)
print(type(new_str))
new_array = []
for item in new_str:
    print(type(item), item)
    if item != "":
        new_array.append(item)

<class 'list'>
<class 'str'> 2к16 это 2016, то есть год. Букву
<class 'str'> к
<class 'str'> пишут вместо нуля.
<class 'str'> 
<class 'str'> 
<class 'str'> Пример выражения:
<class 'str'> 
<class 'str'> - найк аир макс зашквар в 2к16!
<class 'str'> 
<class 'str'> - NBA 2К16 (игра нба 2016)
<class 'str'> 
<class 'str'> - это уже не актуально в 2к16
<class 'str'> 
<class 'str'> 
<class 'str'> dle_media_begin:https://www.youtube.com/watch?v=AfN8a8wX7YM
<class 'str'> 
<class 'str'> dle_media_end
<class 'str'> 
<class 'str'> 
<class 'str'> Можно так же изменять цифру и писать разные года.
<class 'str'> 
<class 'str'> 
<class 'str'> dle_media_begin:https://www.youtube.com/watch?v=5PrUUV9la8o
<class 'str'> 
<class 'str'> dle_media_end
<class 'str'> 
<class 'str'> 
<class 'str'> Уточнение: Буква k в сочетании с числом обозначает кратную приставку кило-, т.е. множитель 1000. Например: warhammer 40000 иногда записывается, как warhammer 40k, или w40k. (Игорь,Витебск)
<class 'str'> 
<class 'str'> 

In [95]:
def remake_fulltext_arrays(fulltext_array):
    new_array = []
    old = literal_eval(str(fulltext_array))
    for item in old[2:]:
        if item != "":
            new_array.append(item)
    return new_array

In [85]:
fullwords_pkl_df['fulltext'] = fullwords_pkl_df['fulltext'].apply(remake_fulltext_arrays)

In [91]:
fullwords_pkl_df.to_csv(vsekidki_fullwords_csv)

In [ ]:
fullwords_df['fulltext'].apply(remake_fulltext_arrays)

In [ ]:
make_fullwords_pkl(vsekidki_fullwords_csv)
add_fullwords_to_pkl(all_words, vsekidki_fullwords_csv)

In [ ]:
def make_fullwords_pkl(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name','fulltext'))


def add_fullwords_to_pkl(dict, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dict:
            writer.writerow((word['name'], word['fulltext'][2:]))

In [9]:
dicton.rename(columns={'story':'origin'}, inplace=True)
dicton.head(5)

,name,definition,type,group,examples,hashtags,origin,synonyms
0,АУЕ,А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. ...,NonNone,NonNone,NonNone,NonNone,"Воряги так говорят, переводится как АРЕСТАНСКИ...",NonNone
1,Патимейкер,Новая песня с незамысловатым текстом. Автор - ...,NonNone,NonNone,NonNone,NonNone,"Патимейкеры это те, кто организовывают тусу (д...",NonNone
2,Аксемора,"Аксиомора ошибочное произнесение слова ""оксюмо...",NonNone,NonNone,NonNone,NonNone,"Компания ""Мегафон"" переозвучила свою рекламу с...",NonNone
3,Голландский штурвал,"Один берет руку другого и начинает управлять, ...",NonNone,NonNone,NonNone,NonNone,"Все пошло из отрывка из фильма ""Зак и Мири Сни...",NonNone
4,Трэшер (Thrasher),"Thrasher — трэшер это скейтерский бренд, котор...",NonNone,NonNone,NonNone,NonNone,NaN,NonNone


In [10]:
all_w = dicton.to_dict('records')

In [13]:
all_w[0]

{'definition': 'А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. Крик, одобрение, кричалка. Что то типо асаб, но новый уровень!',
 'examples': 'NonNone',
 'group': 'NonNone',
 'hashtags': 'NonNone',
 'name': 'АУЕ',
 'origin': 'Воряги так говорят, переводится как АРЕСТАНСКИЙ УКЛАД ЕДИН ИЛИ АРЕСТАНСКОЕ УРКАГАНСКОЕ ЕДИНСТВО. Одним словом, АУЕ это воровское сообщество, которое против законов и ментов.Братва мира благопалучия дому вашему общему процветания ходу воравскому ЖИЗНЬ ВОРАМ (ВЕЧНО)Братва до блогасловит господь мой твой души наши грешные а будь так как сказано ворами ЖИЗНЬ ВОРАМ (ВЕЧНО)кто не снами тот под нами братва если бог за нас то кто против нас?только бляди ЖИЗНЬ ВОРАМ',
 'synonyms': 'NonNone',
 'type': 'NonNone'}

path1 = "vsekidki1.csv"

make_file(path1)
add_words_to_file(all_w, path1)

In [21]:
dicton1 = pd.read_csv(path1)
dicton1.head(5)

,name,definition,type,group,examples,hashtags,origin,synonyms
0,АУЕ,А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. ...,,,,,"Воряги так говорят, переводится как АРЕСТАНСКИ...",
1,Патимейкер,Новая песня с незамысловатым текстом. Автор - ...,,,,,"Патимейкеры это те, кто организовывают тусу (д...",
2,Аксемора,"Аксиомора ошибочное произнесение слова ""оксюмо...",,,,,"Компания ""Мегафон"" переозвучила свою рекламу с...",
3,Голландский штурвал,"Один берет руку другого и начинает управлять, ...",,,,,"Все пошло из отрывка из фильма ""Зак и Мири Сни...",
4,Трэшер (Thrasher),"Thrasher — трэшер это скейтерский бренд, котор...",,,,,NaN,


In [22]:
all_w1 = dicton1.to_dict('records')

In [23]:
all_w1[0]

{'definition': 'А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. Крик, одобрение, кричалка. Что то типо асаб, но новый уровень!',
 'examples': ' ',
 'group': ' ',
 'hashtags': ' ',
 'name': 'АУЕ',
 'origin': 'Воряги так говорят, переводится как АРЕСТАНСКИЙ УКЛАД ЕДИН ИЛИ АРЕСТАНСКОЕ УРКАГАНСКОЕ ЕДИНСТВО. Одним словом, АУЕ это воровское сообщество, которое против законов и ментов.Братва мира благопалучия дому вашему общему процветания ходу воравскому ЖИЗНЬ ВОРАМ (ВЕЧНО)Братва до блогасловит господь мой твой души наши грешные а будь так как сказано ворами ЖИЗНЬ ВОРАМ (ВЕЧНО)кто не снами тот под нами братва если бог за нас то кто против нас?только бляди ЖИЗНЬ ВОРАМ',
 'synonyms': ' ',
 'type': ' '}

In [24]:
path2 = "vsekidki2.csv"
make_file(path2)
add_words_to_file(all_w, path2)

In [25]:
dicton2 = pd.read_csv(path2)
dicton2.head(5)

,name,definition,type,group,examples,hashtags,origin,synonyms
0,АУЕ,А.У.Е расшифровка: Жизнь Ворам Сметь мусорам. ...,,,,,"Воряги так говорят, переводится как АРЕСТАНСКИ...",
1,Патимейкер,Новая песня с незамысловатым текстом. Автор - ...,,,,,"Патимейкеры это те, кто организовывают тусу (д...",
2,Аксемора,"Аксиомора ошибочное произнесение слова ""оксюмо...",,,,,"Компания ""Мегафон"" переозвучила свою рекламу с...",
3,Голландский штурвал,"Один берет руку другого и начинает управлять, ...",,,,,"Все пошло из отрывка из фильма ""Зак и Мири Сни...",
4,Трэшер (Thrasher),"Thrasher — трэшер это скейтерский бренд, котор...",,,,,NaN,


In [ ]:
def Text_from_html_object(text):
    start = text.find("<")
    finish = text.find(">")
    text = text[:start] + text[finish + 1:]
    check_for_tag = text.find('<', start)
    while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
    return text

def Text_from_html_arr(arr):
    answer_text = ''
    for object in arr:
        text = str(object)
        check_for_tag = text.find('<')
        while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
        answer_text += text
    return answer_text

def get_page_count(soup):
    list_of_pages = soup.find_all('a', class_='page_number')
    try:
        last_page = list_of_pages[-2]
    except IndexError:
        return 1
    last_number = Text_from_html_object(str(last_page))
    return int(last_number)

def take_rus_letters_numbers(soup):
    links = []
    list_of_links = soup.find_all('a', class_='alph_menu')
    for link in list_of_links:
        pos = int(str(link).find("%D0%")) + 4
        name = str(str(link)[pos:pos+2])
        if name != "81":
            links.append(name)
    return links

def make_file(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name', 'type', 'group', 'definition', 'examples', 'hashtags', 'story', 'synonyms'))

def add_words_to_file(dict, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dict:
            writer.writerow((word['name'], word['meaning'], word['type'], word['group'], 
                             word['example'], word['story'], word['hashtag'], word['synonyms']))
            

    
# and then rename mycsv.out to mycsv.csv if you wish.

def get_words_from_Teenslang(from_i, to_i, file):
    make_file(file)
    for j in range(from_i, to_i + 1):
        driver.get('http://teenslang.su/content/%D0%'+ rus_letters_numbers[j]) # download site to the Selenium
        html = driver.page_source # saving site source
        soup = BeautifulSoup(html, "lxml")
        page_count = get_page_count(soup)
        print('Буква алфавита № ' + str(j + 1) + ': ')
        dict = []
        for i in range(page_count):
            if i > 0:
                driver.get('http://teenslang.su/?page=' + str(i) + '&content=%D0%'+ rus_letters_numbers[j] + '*') # load sites
                html = driver.page_source # saving site source
                soup = BeautifulSoup(html, "lxml")
            print('Парсинг %d из %d' % (i + 1, page_count))
            table = soup.find('div', class_='visible-xs visible-md visible-sm visible-lg col-lg-6')
            words = table.find_all('div', class_='issueword1')

            for word_row in words:
                word_name = Text_from_html_object(str(word_row.find('span', class_='cap')))
                word_type = Text_from_html_object(str(word_row.find('span', class_='sta')))
                word_group = Text_from_html_object(str(word_row.find('span', class_='sph')))
                word_def = Text_from_html_object(str(word_row.find('span', class_='mea')))
                word_example = Text_from_html_object(str(word_row.find('span', class_='text')))
                word_story = Text_from_html_arr(word_row.find_all('div', class_='category'))
                word_hash = Text_from_html_object(str(word_row.find('span', class_='src')))
                word_syn = Text_from_html_object(str(word_row.find('span', class_='syn')))
                dict.append({
                    'num': i,
                    'name': word_name,
                    'type': word_type,
                    'group': word_group,
                    'meaning': word_def,
                    'example': word_example,
                    'story': word_story,
                    'hashtag': word_hash,
                    'synonyms': word_syn
                })
        add_words_to_file(dict, file)
    print("!!!!!!!!!!!ВСЕ!!!!!!!!!!")
            
        
def get_rus_letters_numbers():
    driver.get("http://teenslang.su/content/%D0%90")
    html = driver.page_source # saving site source
    soup = BeautifulSoup(html, "lxml")
    rus_letters_numbers = take_rus_letters_numbers(soup)
    return rus_letters_numbers

def capitalize_and_strip_string(x):
    return x.capitalize().strip()

In [ ]:
driver = webdriver.PhantomJS()
global driver
rus_letters_numbers = get_rus_letters_numbers()

In [ ]:
file = "dict.csv"

In [ ]:
get_words_from_Teenslang(from_i=9, to_i=9, file=file)

In [ ]:
dict = pd.read_csv(file)
dict.head(5)

In [ ]:
fulldict = pd.read_csv("fullDict.csv")
fulldict.drop_duplicates(keep='first')
fulldict['name'] = fulldict['name'].apply(capitalize_and_strip_string)

In [ ]:
print(fulldict['name'].find('a'))
fulldict[a:a + 3]

In [ ]:
fulldict.to_csv("fullDict.csv", sep=',', index=False)

In [ ]:
wordsNamesSeries = fulldict['name']

In [ ]:
wordsNames.apply(capitalize_and_strip_string)
wordsNames.tolist()

In [ ]:
myDic={'b': 2, 'a': 3, 'c': 1}
def print_dict_sorted_by_value(dictionary):
    for (key, value) in sorted(dictionary.items(), key=lambda x: x[1], reverse=True):
        print(key, ": ", value, sep='')

In [14]:
file = "vsekidki_dict.csv"
driver = webdriver.PhantomJS('/home/kam/Desktop/phantomjs')
global driver
make_file(file)